# Lab 6

In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns 
import matplotlib as mp
import matplotlib.pyplot as plt 

from scipy import stats
from textwrap import wrap

%matplotlib inline
%config InlineBackend.figure_format = 'pdf'

## Import data

In [2]:
KS = pd.read_excel('/Users/kev/Downloads/BCMB 301A Lab Techniques/Lab 6 - Study of reversible inhibition using beta-galactosidase/Data/kevin.xlsx',
                  header=1)

DI = pd.read_excel('/Users/kev/Downloads/BCMB 301A Lab Techniques/Lab 6 - Study of reversible inhibition using beta-galactosidase/Data/Daniel.xlsx',
                  header=13, index_col=1)

TC = pd.read_excel('/Users/kev/Downloads/BCMB 301A Lab Techniques/Lab 6 - Study of reversible inhibition using beta-galactosidase/Data/Tyler H.xlsx',
                  header=1)

## Clean up and analyze data

In [3]:
def abs_ave(D, x0=1, xf=9, dx=10, y0=0, yf=4, n=9, Nanx=[], Nany=[], 
            Inh="Name IDIOT!!!", Unit="Units DUMBASS!!!", I=[0, 10, 20, 40], C=1, V=200):
    # 2D data size
    x = xf-x0
    y = yf-y0
    
    # Turn into 3d array
    # Axis 1 = Substrate
    # Axis 2 = Inhibitor
    # Axis 3 = Time
    A = np.zeros((x, y, n))
    for i in range(0,n):
        a = np.array(D.iloc[x0+dx*i:xf+dx*i, y0:yf])
        A[:,:,i] = a 
    
    # Find change in absorbance with time
    Diff = np.diff(A, axis=2)
    Ave = np.nanmean(Diff, axis=2)
    Ave = pd.DataFrame(Ave)
    
    # Get rid of errors
    if len(Nanx)<1:
        for i in range(0, len(Nanx)):
            Ave.iloc[Nanx[i], Nany[i]] = np.nan

    else: 
        Ave += 0
        
    # Get averages of change with respect to time
    Abs = np.zeros((int(x/2), y))
    for i in range(0, int(x/2)):
        Ave_abs = np.nanmean(Ave.iloc[2*i:2*i+1,:], axis=0)
        Abs[i,:] = Ave_abs

    # Label columns
    Abs = pd.DataFrame(Abs)
    I_c = [i*C/V for i in I]
    Abs.columns = [r'{0}: {1:.3g} {2}'.format(Inh, i, Unit) for i in I_c]
    ABS = Abs.T
    ONPG = [10, 20, 40, 80]
    ABS.columns = [i * 4/200 for i in ONPG]
    return ABS.T

In [4]:
KS_abs = abs_ave(D=KS, Nanx = [0, 7], Nany = [0, 0], Inh='[Sucrose]', Unit='mmoles/l', V=0.2)*6
TC_abs = abs_ave(D=TC, Inh='[Calcium]', C=200, Unit='g/l')*6
DI_abs = abs_ave(D=DI, x0=1, xf=9, y0=1, yf=5, dx=12, Inh='[IPTG]', C=2, Unit='g/l')*6

In [5]:
KS_abs

,[Sucrose]: 0 mmoles/l,[Sucrose]: 50 mmoles/l,[Sucrose]: 100 mmoles/l,[Sucrose]: 200 mmoles/l
0.2,0.26775,0.17775,0.15675,0.14850
0.4,0.23400,0.21075,0.18675,0.15750
0.8,0.25650,0.18900,0.19500,0.16650
1.6,0.27750,0.24675,0.20475,0.16125


## Function to generate graph

In [6]:
# Normal plot 
def plot_norm(D, xname='[ONPG] (mg/ml)', yname=r'$v_0$ ($A_{405}$)'):
    plt.figure(figsize=(10, 5))
    Lines = sns.lineplot(data=D, marker='o',
                         sort=False, # markers=['P', 'o', 'X', 'd'], 
                         palette=['xkcd:lightish green', 'xkcd:medium green', 'xkcd:jungle green', 'xkcd:deep green'],
                         dashes=False, lw=1)
    plt.xlabel(xname, fontsize=12)
    plt.ylabel(yname, fontsize=12)
    plt.xlim(right=2.3)
    plt.legend(loc=1)
    plt.grid()
    plt.show()
    
# Lineweaver-Burk plot
def plot_LB(D, xname=r'$[ONPG]^{-1}$ (ml/mg)', 
            yname=r'$v_0^{-1}$ $\left(A_{405}^{-1}\right)$'):
    A = D.T
    A_name = list(A.columns.values)
    D_name = [1/i for i in A_name]
    A.columns = D_name
    D = 1/(A.T)

    plt.figure(figsize=(10, 5))
    Lines = sns.lineplot(data=D, marker='o',
                         sort=False, # markers=['P', 'o', 'X', 'd'], 
                         palette=['xkcd:lightish green', 'xkcd:medium green', 'xkcd:jungle green', 'xkcd:deep green'],
                         dashes=False, lw=1)
    plt.xlabel(xname, fontsize=12)
    plt.ylabel(yname, fontsize=12)
    plt.xlim(right=np.max(D_name)+2)
    plt.legend(loc=1)
    plt.grid()
    plt.show()

In [7]:
# Plot it out 
# ===========

plot_norm(KS_abs)
plot_norm(TC_abs)
plot_norm(DI_abs)

plot_LB(KS_abs)
plot_LB(TC_abs)
plot_LB(DI_abs)

<Figure size 720x360 with 1 Axes>

<Figure size 720x360 with 1 Axes>

<Figure size 720x360 with 1 Axes>

<Figure size 720x360 with 1 Axes>

<Figure size 720x360 with 1 Axes>

<Figure size 720x360 with 1 Axes>